In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
df_items=pd.read_csv('df_items_limpio.csv')
df_reviews=pd.read_csv('df_reviews_limpio.csv')
df_games=pd.read_csv('df_games_limpio.csv')

### UserData

In [ ]:
df_reviews

In [ ]:
df_items_count = df_items.groupby('user_id')['items_count'].mean().reset_index()
df_items_count.sort_values(by=items)

In [ ]:
df_items

In [ ]:
result=df_reviews[['user_id','recommend']]
df_user_recommend_ratio = result.groupby('user_id')['recommend'].mean().round(2).reset_index()
df_user_recommend_ratio.head(2)

In [7]:
df_usuarios_items = df_items.groupby('user_id')['item_id'].apply(list).reset_index()

In [ ]:
df_usuarios_items.head(2)

In [9]:
df_usuarios_items.to_csv('itemsforuser.csv', index=False)

In [126]:

game_id_to_price = df_games.set_index('id')['price'].to_dict() 

df_usuarios_items['total_spent'] = df_usuarios_items['item_id'].apply(lambda items: sum(game_id_to_price.get(item, 0) for item in items))




In [ ]:
df_gasto=df_usuarios_items.drop('item_id', axis=1)
df_gasto.head(2)

In [128]:
lista=[df_gasto, df_user_recommend_ratio]
temp=df_items_count
for i in range(len(lista)):
    df_user_data= pd.merge(temp,lista[i], on='user_id', how='outer')
    temp=df_user_data
        

In [ ]:
df_user_data.head(2)
df_user_data.loc[:, 'recommend'] = df_user_data['recommend'] * 100
df_user_data.rename(columns={'recommend': 'recommend_ratio'})

In [130]:
df_user_data.to_csv('df_user_data.csv', index=False)

### User Data Función


In [15]:
df_user_data=pd.read_csv('df_user_data.csv')

In [32]:
def userdata(user_id):
    df = df_user_data.loc[df_user_data['user_id'] == f'{user_id}']
    #df.loc[:, 'recommend'] = df['recommend']
    df=df.rename(columns={'recommend': 'porcentaje de recomendacion',
                 'items_count': 'total items', 'total_spent':'total gastado'})

    return df
    

In [ ]:
xd=userdata('--000--')
xd

### Countreviews


In [ ]:
df_reviews.head(2)

In [ ]:
df_count_reviews=df_reviews[['user_id','date']]
df_count_reviews = df_count_reviews.merge(df_user_recommend_ratio, on='user_id', how='left')
df_count_reviews=df_count_reviews.dropna(subset='date')
df_count_reviews.head(2)

In [146]:
df_count_reviews.to_csv('df_count_reviews.csv', index=False)

### Countreviews


In [12]:
df_count_reviews=pd.read_csv('df_count_reviews.csv')

In [13]:
def countreviews(fecha_inicio,fecha_fin):
    df_count_reviews['date'] = pd.to_datetime(df_count_reviews['date'])
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)

    # Filtrar el DataFrame para el rango de fechas deseado
    df_rango = df_count_reviews[(df_count_reviews['date'] >= fecha_inicio) & (df_count_reviews['date'] <= fecha_fin)]
    return [len(df_rango['user_id'].unique()) ,df_rango['recommend'].mean()*100]

In [ ]:
df_rango=countreviews('2012-07-15','2013-07-15')
df_rango

### Género

In [ ]:
df_games.head(2)

In [ ]:
id_to_genre = df_games.dropna(subset='genres').set_index('id')['genres'].to_dict()


In [153]:
def get_all_genres(id_list):
    return id_to_genre.get(id_list,[])

In [154]:
df_items['game_genre']=df_items['item_id'].apply(get_all_genres)

In [ ]:
df_items.sample(2)

In [156]:
user_genre_playtime=defaultdict(int)
for index, row in df_items.iterrows():
    user=row['user_id']
    game=row['item_id']
    playtime=row['playtime_forever']
    genres =eval(id_to_genre.get(game, '[]'))  # Obtener los géneros del juego
    for genre in genres:
        user_genre = (user, genre)
        user_genre_playtime[user_genre] += playtime
    

In [ ]:
user_genre_df = pd.DataFrame(user_genre_playtime.items(), columns=['user_genre', 'playtime'])
user_genre_df.head(10) 

In [ ]:
user_genre_df[['user_id', 'game_genre']] = pd.DataFrame(user_genre_df['user_genre'].tolist(), index=user_genre_df.index)

df_user_genre=user_genre_df.drop(columns='user_genre')
df_user_genre.head(10)

In [ ]:
df_user_genre = df_user_genre.merge(df_items[['user_id','user_url']], on='user_id', how='left')
df_user_genre.sample(n=2)

In [160]:
df_user_genre=df_user_genre.drop_duplicates()

In [161]:
df_user_genre.to_csv('users_playtime_genre.csv', index=False)

# creacion de funcion genre

In [3]:
df_user_genre=pd.read_csv('users_playtime_genre.csv')

In [10]:
def genre(genero:str):
    df=df_user_genre.groupby('game_genre')['playtime'].sum().reset_index()
    df=df.sort_values('playtime', ascending=False).reset_index().drop(columns='index')
    position = df[df['game_genre'] == genero].index[0] + 1 if genero in df['game_genre'].values else None
    position=str(position)
    return position

# creacion de la funcion userforgenre

In [165]:
def userforgenre( genero : str ):
    df = df_user_genre.loc[df_user_genre['game_genre'] == f'{genero}'].sort_values('playtime', ascending=False).reset_index().drop(columns='index')
    return df[['user_id','user_url', 'playtime']].head(5)
    

# DEVELOPER csv

In [168]:
df_developer=df_games[['developer','app_name','price','release_date']].dropna(subset='release_date')
df_developer.sample(10)

,developer,app_name,price,release_date
7273,pinokl games,party hard - dark castle,0.00,2016-09-16
11876,boogygames studios,frontiers.io,0.99,2017-10-12
18375,"smiteworks usa, llc",fantasy grounds - fantastic far east (token pack),4.99,2017-04-03
7362,gritfish,killing time at lightspeed: enhanced edition o...,3.99,2016-09-27
12474,foreverloops gmbh,foreverloops,29.99,2017-11-27
5098,zinertek,fsx: steam edition - natural tree environment ...,11.99,2016-01-12
23288,michael howard,origin of destiny - donation #1,4.99,2016-05-05
20010,sharkbyte studio,ant-gravity: tiny's adventure,12.99,2016-12-15
18003,ronimo games,ted mcpain - awesomenauts character,4.99,2017-04-26
11842,"smiteworks usa, llc",fantasy grounds - pathfinder rpg - hell's rebe...,24.99,2017-10-10


In [169]:
df_developer['year']=df_developer['release_date'].str.extract(r'(\d{4})')
df_developer=df_developer.dropna(subset='year').drop(columns='release_date')

In [170]:
df_developer.sample(3)

,developer,app_name,price,year
1692,"jackbox games, inc.",you don't know jack television,2.99,1997
7598,blind bird,stay close,8.99,2016
28341,sarepta studio,shadow puppeteer,14.99,2014


In [172]:
df_developer.to_csv('df_developer.csv', index=False)

In [173]:
len(df_developer)

29966

# creacion de funcion developer

In [33]:
df_developer=pd.read_csv('df_developer.csv')

In [35]:
def developer(desarrollador: str):
    desarrollador=desarrollador
    desarrollador_lower = desarrollador.lower()
    df = df_developer.loc[df_developer['developer'] == f'{desarrollador_lower}'].sort_values('year')
    df=df.drop_duplicates()
    yearB = 0
    dict_años = defaultdict(list)  # Changed the dictionary value type to list
    items = 0
    items_free=0
    for index, row in df.iterrows():
        year = row['year']
        price = row['price']
        if  year != yearB:
            items = 0  # Restablece la cuenta de elementos si es un nuevo año
            yearB = year
            items_free=0
              
        if price == 0.00:
            items_free+=1
            dict_años[year] = [f'{items_free}', items] 
        else:
            items += 1
            dict_años[year] = [f'{items_free}', items]
    texto=' Free content %'

    df = pd.DataFrame(dict_años.values(), dict_años.keys()).rename(columns={0:f'{desarrollador}{texto}', 1: 'lanzamientos ese año'})
    df[f'{desarrollador}{texto}']= round((pd.to_numeric(df[f'{desarrollador}{texto}']) / df['lanzamientos ese año']*100), 2)
    return df

In [36]:
Temp=developer('capcom')
Temp

### Sentiment

In [ ]:
df_games

In [352]:
df_sentiment_analysis=df_reviews.copy()
df_sentiment_analysis=df_sentiment_analysis[['item_id','review']]
df_sentiment_analysis['review']=df_sentiment_analysis['review'].fillna(1)

In [354]:
df_sentiment_analysis=df_sentiment_analysis.dropna(subset='item_id')

In [ ]:
df_sentiment_analysis

In [357]:
id_to_release = df_games.dropna(subset='id').set_index('id')['release_date'].to_dict()

In [358]:
def id_release(id):
    return id_to_release.get(id,np.nan)

In [359]:
df_sentiment_analysis['año_lanzamiento']=df_sentiment_analysis['item_id'].apply(id_release)

In [360]:
df_sentiment_analysis['año_lanzamiento']=df_sentiment_analysis['año_lanzamiento'].str.extract(r'(\d{4})')

In [ ]:
df_sentiment_analysis

In [362]:
df_sentiment_analysis=df_sentiment_analysis.groupby('año_lanzamiento')['review'].apply(list).reset_index()

In [ ]:
df_sentiment_analysis.head()

In [364]:
df_sentiment_analysis.to_csv('df_sentiment_analysis.csv', index=False)

# Sentiment_analysis

In [38]:
sentiment=pd.read_csv('df_sentiment_analysis.csv')

In [40]:
def sentiment_analysis(año: int):
    df = sentiment.copy()
    filtered_df = df[df['año_lanzamiento'] == año]['review']
    
    if not filtered_df.empty:
        positivo = filtered_df.iloc[0].count('2')
        negativo = filtered_df.iloc[0].count('0')
        neutro = filtered_df.iloc[0].count('1')
        return f'Negative = {negativo}, Neutral = {neutro}, Positive = {positivo}'
    else:
        return 'No hay reseñas para este año de lanzamiento'

In [41]:
Temp3=sentiment_analysis(2016)
Temp3

'Negative = 694, Neutral = 545, Positive = 1759'

In [196]:
positivo= df.count(2)
positivo

44